In [ ]:
import numpy as np


def mat_mul(A, B, b_size):
    m, n = A.shape
    p, q = B.shape

    if n != p:
        print("Matrices multiplication is not possible")
        return

    C = np.zeros((m, q))

    # Perform block matrix multiplication
    for i in range(0, m, b_size):  # row
        for j in range(0, q, b_size):  # col
            for k in range(0, n, b_size):  # inner treat
                block_A = A[i:i + b_size, k:k + b_size]  # change in row
                block_B = B[k:k + b_size, j:j + b_size]  # change in  column

                C[i:i + b_size, j:j + b_size] += np.dot(block_A, block_B)
    return C


In [ ]:
# initialize 4 rows and 4 columns in [0,1) interval

# A = np.random.rand(4, 4)
# B = np.random.rand(4, 4)

In [ ]:
# -----------main program --------------

A = np.array([
    [1, 2, 4, 1],
    [1, 0, -1, -2],
    [2, -1, 3, 1],
    [1, 2, 3, 4]
])
B = np.array([
    [-2, 2, -3, 3],
    [0, 1, -1, 2],
    [-2, -1, 0, 1],
    [4, 0, 1, 4]
])



ans = mat_mul(A, B, 1)
print(ans)

[[-6.  0. -4. 15.]
 [-8.  3. -5. -6.]
 [-6.  0. -4. 11.]
 [ 8.  1. -1. 26.]]
